In [2]:
import folium
import pandas as pd

In [22]:
import pandas as pd
import pydeck as pdk
import geopandas as gpda
import config
import json
import requests
import sys
import folium

path = "C:\github/aide-analysis/"
sys.path.append(path)

from core.config import Config
config = Config()


In [28]:
# Vworld 시군구 경계 API 호출
def gl(sig_cd):
    # 엔드포인트
    endpoint = "http://api.vworld.kr/req/data"

    # API 파라미터
    service = "data"
    request = "GetFeature"

    # 행정동 경계
    data = "LT_C_ADSIGG_INFO"

    # 요청 파라미터
    key = config.OPEN_API["vworld"]
    page = 1
    size = 1000
    
    attrFilter = f"sig_cd:like:{sig_cd}"
    url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"

    # 요청 결과
    res = json.loads(requests.get(url).text)

    # GeoJson 생성
    featureCollection = {"type": "FeatureCollection"}
    features = []
    while True:
        # 요청 URL
        url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"
        # 요청 결과
        res = json.loads(requests.get(url).text)
        # GeoJson 추가
        features = features + res["response"]["result"]["featureCollection"]["features"]
        # 페이지
        page_current = int(res["response"]["page"]["current"])
        page_total = int(res["response"]["page"]["total"])
        page += 1
        if page_current == page_total:
            break
    featureCollection["features"] = features
    geo = featureCollection
    return geo
geo = gl(11650)

In [30]:
# params
center = [37.541, 126.986]
lon = 127.0080061098534
lat = 37.50989980338221

# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    width = 750,
    height = 500
)


# Marker(location : 마커를 추가할 경위도 좌표/ popup : 표기할 팝업문구(클릭시 문구)/ tooltip : 표기할 툴팁 지정(마우스 오버))
folium.Marker(
    location = [lat, lon],
    popup = '반포자이',
    tooltip = '반포자이',
    icon = folium.Icon('red', icon = 'star')
).add_to(m)

# Circle
# folium.CircleMarker(
#     location = [lat,lon],
#     color = 'tomato',
#     radius = 200,
#     tooltip = '반포자이 인근'
# ).add_to(m)

# GeoJson 활용
folium.GeoJson(
    geo,
    name = '서초구'
).add_to(m)

m